In [1]:
import flask
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlalchemy

In [ ]:
# serve.py

from flask import Flask, redirect, url_for, request
from flask import render_template

engine = create_engine("sqlite:///project.db")
# creates a Flask application, named app
app = Flask(__name__)

# a route where we will display a welcome message via an HTML template
@app.route("/")
def init():  
    return render_template('table.html', x=0)


@app.route('/actors')
def asb():
    return str(engine.execute("select * from Bedding Where beds=3").fetchall());

@app.route('/xyz', methods = ['POST', 'GET'])
def xyz():
    if request.method == 'POST':
        x = request.form["x"]

        return redirect(url_for('abc', x=x,y=y,z=z))
    
@app.route('/predefined', methods = ['POST', 'GET'])
def select():
    if request.method == 'POST':
        x = request.form["query_nr"]
        return redirect(url_for('table', x=x))
    
@app.route('/abc/<x>/<y>/<z>')
def abc(x,y,z):
    return render_template('index.html', message=str(engine.execute("select * from "+x+" Where "+y+" = "+z).fetchall()))

@app.route("/table/<x>")
def table(x):
    list_of_predefined = ["",
                          "SELECT AVG(P.price) FROM Listing L, Pricing P WHERE L.id=P.id AND L.beds=8 ;", 
                          "SELECT AVG(review_scores_cleanliness) FROM Listing L, Amenities A, SCORE S, PROVIDE P WHERE L.id=S.id AND P.id=L.id AND P.amenities='TV' ;"
                         "SELECT DISTINCT L.host_id FROM Listing L, AVAILABLE_AT AV WHERE AV.id=L.id AND AV.available='t' AND AV.date <= DATE '2019-09-31' AND AV.date >= DATE '2019-03-00' ;"] 
    test = engine.execute(list_of_predefined[int(x)]).fetchall();
    columns=engine.execute(list_of_predefined[int(x)]).keys();
    count=len(engine.execute(list_of_predefined[int(x)]).keys());
    return render_template('table.html', test=test, columns=columns, count= count)
# run the application
if __name__=="__main__":
     app.run(port=5000, debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Apr/2019 20:26:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2019 20:26:57] "POST /predefined HTTP/1.1" 302 -
127.0.0.1 - - [29/Apr/2019 20:26:57] "GET /table/1 HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2019 20:27:01] "POST /predefined HTTP/1.1" 302 -
[2019-04-29 20:27:01,213] ERROR in app: Exception on /table/2 [GET]
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/sqlalchemy/engine/base.py", line 1193, in _execute_context
    context)
  File "/anaconda3/lib/python3.6/site-packages/sqlalchemy/engine/default.py", line 509, in do_execute
    cursor.execute(statement, parameters)
sqlite3.OperationalError: no such table: PROVIDE

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/flask/app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "/anaconda3/li